# Backcasting Demo Notebook

_Loren Champlin_

Adapted from _Adarsh Pyarelal_'s WM 12 Month Evaluation Notebook 

As always, we begin with imports, and print out the commit hash for a rendered
version of the notebook.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pickle
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
from delphi.visualization import visualize
import delphi.jupyter_tools as jt
import numpy as np
import pandas as pd
from scipy import stats
#Comment out the next line if you do not have the delphi.db file. 
from delphi.db import engine
jt.print_commit_hash_message()
import random as rm
import delphi.evaluation as EN
import warnings
#warnings.filterwarnings("ignore")
import logging
logging.getLogger().setLevel(logging.CRITICAL)
from indra.statements import (
    Concept,
    Influence,
    Evidence,
    Event,
    QualitativeDelta,
)
from delphi.utils.indra import *
from delphi.utils.shell import cd
import seaborn as sns
import matplotlib.pyplot as plt
from delphi.utils.fp import flatMap, take, ltake, lmap, pairwise, iterate, exists
from delphi.cpp.AnalysisGraph import AnalysisGraph as AG

This notebook has been rendered with commit b'e1b345d5d16b1d557491defcee67647fc655a308' of Delphi.


Here I will set random seeds

In [2]:
np.random.seed(87)
rm.seed(87)

In [3]:
statements = [ (("large", -1, "UN/entities/human/financial/economic/inflation"),("small", 1, "UN/events/human/human_migration"))]


Now we load the Causal Analysis Graph (CAG). This is CAG was inferred by reading in a JSON corpus and was pruned and adjusted to be human migration centered. Also is a list of the nodes contained in the CAG

In [4]:
#with open("../scripts/build/migration_centered_CAG.pkl",'rb') as f:
    #G = pickle.load(f)

G = AG.from_statements(statements)

None


Next we map indicator variables to nodes. For the most part indicator variables can be inferred from available data and texts, but we can also manually map indicators to nodes. There is also a list of the indicator variables in the same order as the list of nodes above (i.e "Claims on other sectors of the domestic economy" is attached to "UN/events/human/economic_crisis".

In [6]:
query = " ".join(
        [
            f"select * from indicator",
            f"where `Variable` like 'New asylum seeking applicants'",
        ]
    )

results = engine.execute(query)

pd.DataFrame(results, columns=results.keys())

,index,Country,County,Month,Source,State,Unit,Value,Variable,Year
0,15586,South Sudan,None,7.0,UNHCR,None,applicants,1,New asylum seeking applicants,2011
1,15587,South Sudan,None,9.0,UNHCR,None,applicants,1,New asylum seeking applicants,2011
2,15588,South Sudan,None,11.0,UNHCR,None,applicants,1,New asylum seeking applicants,2011
3,15589,South Sudan,None,12.0,UNHCR,None,applicants,3,New asylum seeking applicants,2011
4,15590,South Sudan,None,1.0,UNHCR,None,applicants,4,New asylum seeking applicants,2012
5,15591,South Sudan,None,2.0,UNHCR,None,applicants,2,New asylum seeking applicants,2012
6,15592,South Sudan,None,3.0,UNHCR,None,applicants,8,New asylum seeking applicants,2012
7,15593,South Sudan,None,4.0,UNHCR,None,applicants,1,New asylum seeking applicants,2012
8,15594,South Sudan,None,5.0,UNHCR,None,applicants,1,New asylum seeking applicants,2012
9,15595,South Sudan,None,6.0,UNHCR,None,applicants,4,New asylum seeking applicants,2012
